In [9]:
using Pkg; Pkg.activate(".");
using Revise
using Statistics
import MUST
using PyPlot

MUST.@get_help_py stagger
MUST.@get_help visual 
MUST.@import_dispatch "/shared-storage/eitner/model_grid/dispatch2/";

  Activating environment at `/usr/local/autofs.shared/shared-storage/eitner/model_grid/MUST.jl/examples/stagger2bifrost/Project.toml`


In [10]:
folder = "stagger_sun/"
model  = "t5777g44m0005";
snap_j = MUST.StaggerSnap(model, folder);

┌ Warning: bx not present.
└ @ MUST /usr/local/autofs.shared/shared-storage/eitner/model_grid/MUST.jl/src/_stagger.jl:332
┌ Warning: by not present.
└ @ MUST /usr/local/autofs.shared/shared-storage/eitner/model_grid/MUST.jl/src/_stagger.jl:332
┌ Warning: bz not present.
└ @ MUST /usr/local/autofs.shared/shared-storage/eitner/model_grid/MUST.jl/src/_stagger.jl:332


In [11]:
# The gas pressure is not saved in dispatch, so we use the same EOS for both to get it
eos_path = "input_data/solar_lte"
eos_path = "input_data/TSO_sun_Magg_v1"
eos_path = "input_data/TSO_sun_Magg_v1.1"
sqEOS = MUST.SquareGasEOS(MUST.@in_dispatch(eos_path));

In [12]:
b = MUST.Box(snap_j, eos=sqEOS);

In [13]:
z = MUST.axis(b, :z)
d = MUST.plane_statistic(mean, b, :τ_ross)
plt.semilogy(z, d)

1-element Vector{PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x2aafc808fd30>

In [14]:
MUST.save(b; name="box_solar_stagger_TSOv1.1", folder="")

"box_solar_stagger_TSOv1.1.hdf5"

In [15]:
stagger_τ  = MUST.height_scale(b, :τ_ross);

In [16]:
MUST.save(stagger_τ; name="box_solar_stagger_TSOv1.1_t", folder="");